In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/dataset/train_data.csv", index_col='Id')
train

In [ ]:
train.describe()

<p>Tentando ver quais colunas possuem valores faltantes: </p>

In [ ]:
(train == '?').any()

<p> E a quantidade de valores falntantes em cada coluna:</p>

In [ ]:
train[train == '?'].count()

In [ ]:
train['workclass'].value_counts().plot(kind = 'bar')

In [ ]:
train['occupation'].value_counts().plot(kind = 'bar')

In [ ]:
train['native.country'].value_counts().plot(kind = 'bar')

<p> Para as colunas 'workclass' e 'native.country' há uma tendência esmagadora de os indivíduos pertencerem ao setor privado e possuírem a nacionalidade americana. Nessas duas colunas, assumirei que os valores faltantes corresponderiam a esses dois valores.</p>

In [ ]:
train['workclass'] = train['workclass'].replace(to_replace = '?', value = 'Private')

In [ ]:
train['native.country'] = train['native.country'].replace(to_replace = '?', value = 'United-States')

In [ ]:
train[train == '?'].count()

<p> A coluna 'occupation' porém é muito mais variada, e assim a probabilidade de pertencer a uma certa categoria é menor. Irei então desconsiderar seus missing values. </p>

In [ ]:
train = train.loc[train.occupation != '?']

In [ ]:
train

In [ ]:
Xtrain = train.loc[:,'age':'native.country']
Ytrain = train.income
Xtrain.head()

In [ ]:
Ytrain.head()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

<p> Tenho ainda que definir como lidar com strings no knn. Usarei a função get_dummies do pandas para criar um novo DataFrame que codifica as diferentes categorias de strings em uma coluna cada.</p>

In [ ]:
from sklearn import preprocessing

In [ ]:
Xtrain = pd.get_dummies(Xtrain)
Xtrain

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 10)
scores = cross_val_score(knn, Xtrain, Ytrain, cv=10)

In [ ]:
scores.mean()

In [ ]:
scores_array = []
for i in range(1,25):
    knn = KNeighborsClassifier(n_neighbors = i)
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=10)
    scores_array.append(scores.mean())
    


In [ ]:
plt.plot(scores_array, 'ro')

<p> 

<p> O resultado anterior mostra que o KNN nesse caso tende ao melhor em n=21</p>

In [ ]:
test = pd.read_csv('../input/dataset/test_data.csv', na_values='?', index_col='Id')
test

In [ ]:
Xtest = pd.get_dummies(test)
Xtest.head()

In [ ]:
##garantindo que ambas tenham a mesma dimensão
missing_cols = set( Xtrain.columns ) - set( Xtest.columns )
for c in missing_cols:
    Xtest[c] = 0
Xtest = Xtest[Xtrain.columns]

In [ ]:
knn = knn = KNeighborsClassifier(n_neighbors = 21)
knn.fit(Xtrain,Ytrain)

In [ ]:
YtestPred = knn.predict(Xtest)
YtestPred

In [ ]:
prediction = pd.DataFrame(index = test.index)
prediction['income'] = YtestPred

In [ ]:
prediction

In [ ]:
prediction.to_csv("submition.csv")